### Sentiment Analysis with Transformers

In [ ]:
%env PIP_DISABLE_PIP_VERSION_CHECK=1

In [ ]:
!pip install -q -U watermark
!pip install -q spacy
!pip install -q tensorflow
!pip install -q nltk transformers
!pip install -q keras tf-keras keras-preprocessing

In [ ]:
%env TF_CPP_MIN_LOG_LEVEL=3
%env TF_ENABLE_ONEDNN_OPTS=0

In [ ]:
# 1. Imports
import os
import math
import nltk
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from tokenizers import BertWordPieceTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.metrics import Precision, Recall, AUC
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, LearningRateScheduler, CallbackList, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from keras.saving import register_keras_serializable
from tensorflow.keras.layers import Layer, Dense
from transformers import TFDistilBertModel, DistilBertConfig
from tensorflow.keras.metrics import Precision, Recall, AUC
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('error', category=DeprecationWarning)

In [ ]:
%reload_ext watermark
%watermark -a "shortthirdman"

In [ ]:
if os.environ['TF_ENABLE_ONEDNN_OPTS'] != '0':
    print("TensorFlow has not been correctly")

In [ ]:
# 2.a Loading Training Data
training_data = pd.read_csv('../../data/training_data.txt', header=None, delimiter=';')

# 2.b Loading Test Data
test_data = pd.read_csv('../../data/test_data.txt', header=None, delimiter=';')

In [ ]:
# 3. Adjusting Column Names
training_data = training_data.rename(columns={0: 'text', 1: 'sentiment'})
test_data = test_data.rename(columns={0: 'text', 1: 'sentiment'})

In [ ]:
# 4. Checking Dataset Shape
training_data.shape

In [ ]:
# 5. Checking Test Dataset Shape
test_data.shape

In [ ]:
# 6. Training Data Sample
training_data.head()

In [ ]:
# 7. Sentiments in Training Data
training_data['sentiment'].value_counts()

In [ ]:
# 8. Sentiments in Test Data
test_data['sentiment'].value_counts()

In [ ]:
!python -m spacy download en_core_web_md -q

In [ ]:
# 9. Load SpaCy Model
nlp = spacy.load('en_core_web_md')

In [ ]:
# 10. Definition of the 'preprocess_text' Function, Which Takes a Text as a Parameter
def preprocess_text(text):

    # 10.a Process the text using the SpaCy model
    doc = nlp(text)

    # 10.b Create a list of lemmatized tokens, converted to lowercase, stripped of whitespace,
    # excluding stopwords
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop]

    # 10.c Return the processed tokens as a single string, joined with spaces
    return ' '.join(tokens)

In [ ]:
%%time
# 11. Apply the Preprocessing Function to Training Data
training_data['processed_text'] = training_data['text'].apply(preprocess_text)

# 12. Apply the Preprocessing Function to Test Data
test_data['processed_text'] = test_data['text'].apply(preprocess_text)

In [ ]:
# 13. Data Sample
training_data.head()

#### Model Implementation: Vectorization with TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) is a technique that has been used for a long time in natural language processing.

In [ ]:
# 14. Create the Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [ ]:
# 15. Apply the Vectorizer
train_tfidf = tfidf_vectorizer.fit_transform(training_data['processed_text'])
test_tfidf = tfidf_vectorizer.transform(test_data['processed_text'])

In [ ]:
# 16. Check Shape of Training Data TF-IDF Matrix
train_tfidf.shape

In [ ]:
# 17. Check Type of Training Data TF-IDF Matrix
type(train_tfidf)

In [ ]:
# 18. Convert Input Data (Text) to Array
X_train_array = train_tfidf.toarray()
X_test_array = test_tfidf.toarray()

In [ ]:
# 19. Create the Label Encoder
label_encoder_v1 = LabelEncoder()

In [ ]:
# 20. Fit and Transform the Target Variable in Training Data
y_train_le = label_encoder_v1.fit_transform(training_data['sentiment'])

# 21. Transform the Target Variable in Test Data
y_test_le = label_encoder_v1.transform(test_data['sentiment'])

In [ ]:
# 22. Compute Class Weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_le), y=y_train_le)

In [ ]:
# 24. Split Data into Training and Validation Sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_array,
    y_train_le,
    test_size=0.2,
    random_state=42,
    stratify=y_train_le
)

In [ ]:
# 25. Convert Target Variable to Categorical Type
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test_le)
y_val_encoded = to_categorical(y_val)

In [ ]:
# 26. Check Shape of Encoded Target Variables
y_train_encoded.shape, y_test_encoded.shape, y_val_encoded.shape

In [ ]:
# 27. Create the Model

# 27.a Initialize a sequential model. Sequential models are a linear stack of layers.
model_v1 = Sequential()

# 27.b Add the first dense (fully-connected) layer to the model
model_v1.add(Dense(
    4096,
    activation='selu',  # Use the SELU (Scaled Exponential Linear Unit) activation function
    kernel_initializer='lecun_normal',  # Initialize weights with LeCun normal distribution
    input_shape=(X_train.shape[1],),  # Define input shape based on the number of features in X_train
    kernel_regularizer=tf.keras.regularizers.l2(0.01)  # Apply L2 regularization to reduce overfitting
))

# 27.c Add the second dense layer
model_v1.add(Dense(
    2048,
    activation='selu',
    kernel_initializer='lecun_normal',
    kernel_regularizer=tf.keras.regularizers.l2(0.01)
))

# 27.d Add the third dense layer
model_v1.add(Dense(
    1024,
    activation='selu',
    kernel_initializer='lecun_normal',
    kernel_regularizer=tf.keras.regularizers.l2(0.1)
))

# 27.e Add the fourth dense layer
# Layer with 64 neurons and SELU activation
model_v1.add(Dense(64, activation='selu'))

# 27.f Add the output layer
# Output layer with 6 neurons and softmax activation for multi-class classification
model_v1.add(Dense(6, activation='softmax'))

In [ ]:
# 28. Assign Specific Weights to the Bias Vector of the Model's Last Layer
model_v1.layers[-1].bias.assign(class_weights)

In [ ]:
# 29. Compile the Model

# 29.a Define the optimizer as 'Adam'.
# Adam is an optimization algorithm used as an alternative to the classic stochastic gradient descent
# procedure, updating network weights iteratively based on training data.

# 29.b Set the loss function to 'categorical_crossentropy'.
# This is suitable for multi-class classification problems where labels are provided in one-hot encoded format.

# 29.c Specify the evaluation metrics for the model as 'accuracy', along with Precision, Recall, and AUC.
# Accuracy is a common metric to evaluate classification model performance.
model_v1.compile(
    optimizer='Adam',
    loss=tf.losses.categorical_crossentropy,
    metrics=['accuracy', Precision(), Recall(), AUC()]
)

In [ ]:
# 30. Display Model Summary
model_v1.summary()

In [ ]:
# 31. Learning Rate Scheduler Function
def step_decay(epoch):
    
    # 31.a Initial learning rate
    initial_lrate = 0.001
    
    # 31.b Drop factor for learning rate decay
    drop = 0.5
    
    # 31.c Number of epochs after which learning rate is reduced
    epochs_drop = 10.0
    
    # 31.d Calculate the updated learning rate
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    
    return lrate

In [ ]:
# 32. Learning Rate Scheduler
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# 33. Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    restore_best_weights=True,  # Restore the model weights from the epoch with the best validation loss
    patience=3  # Stop training after 3 epochs with no improvement
)

In [ ]:
# 34. Hyperparameters
num_epochs = 20  # Number of epochs
batch_size = 256  # Batch size

In [ ]:
%%time
# 35. Model Training
history = model_v1.fit(
    X_train,  # Training data
    y_train_encoded,  # Encoded training labels
    validation_data=(X_val, y_val_encoded),  # Validation data and labels
    epochs=num_epochs,  # Number of epochs
    batch_size=batch_size,  # Batch size
    callbacks=[early_stopping, lr_scheduler]  # Callback functions: early stopping and learning rate scheduler
)

In [ ]:
# 36. Extract Training and Validation Loss
loss, val_loss = history.history['loss'], history.history['val_loss']

In [ ]:
# 37. Plot Training and Validation Loss
plt.plot(loss, label='Training Loss')  # Plot training loss
plt.plot(val_loss, label='Validation Loss')  # Plot validation loss
plt.legend()  # Add a legend
plt.show()  # Display the plot

In [ ]:
# 38. Predictions on Test Data
predictions_v1 = model_v1.predict(X_test_array)

In [ ]:
# 39. Extract Predicted Labels
predictions_v1_labels = predictions_v1.argmax(axis=1)

In [ ]:
# 40. Print Classification Report
print(classification_report(y_test_le, predictions_v1_labels))

In [ ]:
# 41. Print Confusion Matrix
print(confusion_matrix(y_test_le, predictions_v1_labels))

In [ ]:
# 42. Print Accuracy Score
print(accuracy_score(y_test_le, predictions_v1_labels))

In [ ]:
# 43. Save the Model
model_v1.save('model_v1.keras')

In [ ]:
# 44. Load the Model
loaded_model = load_model('model_v1.keras')

In [ ]:
# 45. New Sentence (sentiment = fear)
sentence = "i even feel a little shaky"

In [ ]:
# 46. Create a DataFrame with the Sentence
df_new = pd.DataFrame({'Sentence': [sentence]})

In [ ]:
# 47. Apply the Preprocessing Function
df_new['Processed_Sentence'] = df_new['Sentence'].apply(preprocess_text)

In [ ]:
# 48. Display the DataFrame
df_new

In [ ]:
# 49. Apply Vectorization
df_new_tfidf = tfidf_vectorizer.transform(df_new['Processed_Sentence'])

In [ ]:
# 50. Convert to Array
df_new_array = df_new_tfidf.toarray()

In [ ]:
# 51. Predictions
predictions = loaded_model.predict(df_new_array)

In [ ]:
# 52. Display Predictions
predictions

In [ ]:
# 53. Select the Class with the Highest Probability
highest_prob_class = np.argmax(predictions, axis=1)

# 54. Display the Class with the Highest Probability
highest_prob_class

In [ ]:
# 55. Get the Class Name
class_name = label_encoder_v1.inverse_transform(highest_prob_class)

# 56. Predicted Class
class_name

#### Model Implementation: LSTM (Long Short-Term Memory)

The LSTM is excellent for tasks in Natural Language Processing. It also performs well with time series data, as it works with sequential inputs. These sequences can be text data (sentences) or, for example, time series data.

However, LSTM has a significant limitation: it struggles with long-term context. During training, it loses information over the long term due to the vanishing gradient problem. As the gradient diminishes, it eventually disappears, limiting its capacity for larger datasets or extended contexts.

If the goal is for the model to learn something extensive, such as the entire content of a book, LSTM won’t suffice. For that, you need the Transformer.

In [ ]:
# 58. Create the Tokenizer
tokenizer = Tokenizer()

In [ ]:
# 59. Fit the Tokenizer with Processed Texts
tokenizer.fit_on_texts(training_data['processed_text'])

In [ ]:
# 60. Extract Word Index
word_index = tokenizer.word_index

In [ ]:
# 61. Check the Length of the Word Index
len(word_index)

In [ ]:
# 62. Iterate Over Key-Value Pairs in the Dictionary
for i, (key, value) in enumerate(word_index.items()):
    print(key, value)
    # Break the loop after printing 10 items
    if i == 9:
        break

In [ ]:
# 63. Convert Training Texts to Token Sequences
train_sequences = tokenizer.texts_to_sequences(training_data['processed_text'])

In [ ]:
# 64. Define the Maximum Length of Sequences
max_length = 100

In [ ]:
# 65. Padding Training Sequences
train_sequences_padded = pad_sequences(train_sequences, maxlen=max_length, truncating='post')

In [ ]:
# 66. Convert Test Texts to Token Sequences
test_sequences = tokenizer.texts_to_sequences(test_data['processed_text'])

In [ ]:
# 67. Padding Test Sequences
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_length)

In [ ]:
# 68. Create the Label Encoder
label_encoder_v2 = LabelEncoder()

In [ ]:
# 69. Fit and Transform Sentiment Labels for Training
y_train_le = label_encoder_v2.fit_transform(training_data['sentiment'])

In [ ]:
# 70. Transform Sentiment Labels for Testing
y_test_le = label_encoder_v2.transform(test_data['sentiment'])

In [ ]:
# 71. Convert Labels to Categorical Variables
y_train_encoded = to_categorical(y_train_le)
y_test_encoded = to_categorical(y_test_le)

In [ ]:
# 72. Define Vocabulary Size
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# 73. Print Vocabulary Size
print(vocab_size)

In [ ]:
# 74. Define Embedding Dimension
embedding_dim = max_length

In [ ]:
# 75. Construct the LSTM Model
model_v2 = tf.keras.Sequential([
   
    # 75.a Embedding layer with the vocabulary size, embedding dimension, and input length
    Embedding(vocab_size, embedding_dim, input_length=max_length),

    # 75.b Bidirectional LSTM layer with 64 units
    Bidirectional(LSTM(64)),

    # 75.c Dropout layer to prevent overfitting
    Dropout(0.4),

    # 75.d Dense layer with 32 units, Leaky ReLU activation, and L1/L2 regularization
    Dense(32, activation='leaky_relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),

    # 75.e Additional Dropout layer
    Dropout(0.4),

    # 75.f Output layer with 6 units and softmax activation for multi-class classification
    Dense(6, activation='softmax')
])

In [ ]:
# 76. Compile the Model
model_v2.compile(
    loss='categorical_crossentropy',  # Loss function for multi-class classification
    optimizer='adam',  # Adam optimizer
    metrics=['accuracy', Precision(), Recall(), AUC()]  # Evaluation metrics
)

In [ ]:
# 77. Display Model Summary
print(model_v2.summary())

In [ ]:
# 78. Define Input Data as Array
input_data = np.array(train_sequences_padded)

In [ ]:
# 79. Define Output Data as Array
output_data = np.array(y_train_encoded)

In [ ]:
# 80. Define Hyperparameters
num_epochs = 35  # Number of epochs
validation_split_value = 0.2  # Validation split percentage
patience = 5  # Early stopping patience

In [ ]:
# 81. Configure Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience)

In [ ]:
# 82. Train the Model
%%time
history = model_v2.fit(
    input_data,  # Training input data
    output_data,  # Training output data
    epochs=num_epochs,  # Number of epochs
    verbose=1,  # Verbosity level
    validation_split=validation_split_value,  # Validation split ratio
    callbacks=[early_stopping]  # Callback for early stopping
)

In [ ]:
# 83. Plot Error Curves
loss, val_loss = history.history['loss'], history.history['val_loss']  # Extract training and validation loss

plt.plot(loss, label='Training Error')  # Plot training error
plt.plot(val_loss, label='Validation Error')  # Plot validation error

plt.legend()  # Add a legend
plt.show()  # Display the plot

In [ ]:
# 84. Predictions on Test Data
predictions = model_v2.predict(test_sequences_padded)

In [ ]:
# 85. Determine Predicted Labels
predicted_labels = predictions.argmax(axis=1)

In [ ]:
# 86. Display Classification Report
print(classification_report(y_test_le, predicted_labels))

In [ ]:
# 87. Display Confusion Matrix
print(confusion_matrix(y_test_le, predicted_labels))

In [ ]:
# 88. Display Model Accuracy
print(accuracy_score(y_test_le, predicted_labels))

In [ ]:
# 89. Save the Model
model_v2.save('model_v2.keras')

In [ ]:
# 90. Load the Saved Model
loaded_model = load_model('model_v2.keras')

In [ ]:
# 91. New Sentence (Sentiment = Fear)
sentence = "i even feel a little shaky"

In [ ]:
# 92. Create a DataFrame with the Sentence
df_new = pd.DataFrame({'Sentence': [sentence]})

In [ ]:
# 93. Apply the Preprocessing Function
df_new['Processed_Sentence'] = df_new['Sentence'].apply(preprocess_text)

In [ ]:
# 94. Process New Data
new_sequences = tokenizer.texts_to_sequences(df_new['Processed_Sentence'])
new_sequences_padded = pad_sequences(new_sequences, maxlen=max_length)

In [ ]:
# 95. Make Predictions with the Loaded Model
predictions = loaded_model.predict(new_sequences_padded)

In [ ]:
# 96. Select the Class with the Highest Probability
highest_prob_class = np.argmax(predictions, axis=1)

# 97. Display the Class with the Highest Probability
highest_prob_class

In [ ]:
# 98. Get the Class Name
class_name = label_encoder_v2.inverse_transform(highest_prob_class)

# 99. Predicted Class
class_name

In [ ]:
# 100. Display Data Processed with SpaCy
training_data.head()

In [ ]:
# 101. Function to Encode Text into Integer Sequences for BERT Input
def encode_texts(texts, tokenizer, chunk_size=256, maxlen=512):

    # 101.a Configure the tokenizer to truncate texts to the specified maximum length
    tokenizer.enable_truncation(max_length=maxlen)

    # 101.b Configure the tokenizer to apply padding up to the specified maximum length
    tokenizer.enable_padding(length=maxlen)

    # 101.c List to store input IDs generated by the tokenizer
    input_ids = []

    # 101.d List to store attention masks generated by the tokenizer
    attention_masks = []

    # 101.e Iterate over the texts in chunks of the specified chunk_size
    for i in tqdm(range(0, len(texts), chunk_size)):

        # 101.f Select a chunk of texts to process
        text_chunk = texts[i:i+chunk_size].tolist()

        # 101.g Encode the text chunk in batches using the tokenizer
        encs = tokenizer.encode_batch(text_chunk)

        # 101.h Add the encoded input IDs to the input_ids list
        input_ids.extend([enc.ids for enc in encs])

        # 101.i Add the generated attention masks to the attention_masks list
        attention_masks.extend([enc.attention_mask for enc in encs])

    # 101.j Return the input IDs and attention masks as numpy arrays
    return np.array(input_ids), np.array(attention_masks)

In [ ]:
# 102. Load the Pre-Trained Model's Tokenizer
bert_tokenizer = transformers.DistilBertTokenizer.from_pretrained(
'distilbert-base-multilingual-cased')

In [ ]:
# 103. Save the Tokenizer and Vocabulary Locally
bert_tokenizer.save_pretrained('.')

In [ ]:
# 104. Load a Faster Tokenizer Using the Main Tokenizer's Vocabulary
from tokenizers import BertWordPieceTokenizer
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)

In [ ]:
# 105. Visualize the Tokenizer
fast_tokenizer

In [ ]:
# 106. Split Data into Training and Validation Sets with Stratified Sampling

# Processed text for training
X_train, X_valid, Y_train, Y_valid = train_test_split(
    training_data['processed_text'].values,  

    # Sentiment labels for training
    training_data['sentiment'].values,  

    # Validation split ratio
    test_size=0.2,  

    # Random state for reproducibility
    random_state=42,  

    # Stratified sampling by sentiment labels
    stratify=training_data['sentiment']  
)

In [ ]:
# 107. Define Maximum Length for Texts
max_length = 100

In [ ]:
# 108. Apply Encoding (Tokenization) to the Data
X_train_final, train_mask = encode_texts(X_train, fast_tokenizer, maxlen=max_length)
X_valid_final, valid_mask = encode_texts(X_valid, fast_tokenizer, maxlen=max_length)
X_test_final, test_mask = encode_texts(test_data['processed_text'].to_numpy(), fast_tokenizer, maxlen=max_length)

In [ ]:
# 109. Check Shape of Final Training Data
X_train_final.shape

In [ ]:
# 110. Define the Encoder for Output Data
label_encoder_v3 = LabelEncoder()

In [ ]:
# 111. Apply the Encoder (fit_transform only on Training Data)
y_train_le = label_encoder_v3.fit_transform(Y_train)
y_valid_le = label_encoder_v3.transform(Y_valid)
y_test_le = label_encoder_v3.transform(test_data['sentiment'])

In [ ]:
# 112. Convert Output Variable to Categorical
y_train_encoded = to_categorical(y_train_le)
y_valid_encoded = to_categorical(y_valid_le)
y_test_encoded = to_categorical(y_test_le)

In [ ]:
# 113. Define Batch Size
BATCH_SIZE = 16

In [ ]:
# 114. Prepare the Training Dataset in the Format Expected by TensorFlow

# Combine inputs and labels
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices(((X_train_final, train_mask), y_train_encoded))  

    # Repeat the dataset for multiple epochs
    .repeat()  

    # Shuffle the data with a buffer size of 2048
    .shuffle(2048)  

    # Group data into batches of the specified size
    .batch(BATCH_SIZE)  
)

In [ ]:
# 117. Function to Create the Model
def create_model(transformer, max_len=512):

    # 117.a Input layer for word IDs
    input_word_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="input_word_ids"
    )

    # 117.b Input layer for attention masks
    attention_mask = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="attention_mask"
    )

    # 117.c Custom layer for the Transformer
    sequence_output = TransformerLayer(transformer)(
        [input_word_ids, attention_mask]
    )

    # 117.d Select the CLS token (first token)
    cls_token = sequence_output[:, 0, :]

    # 117.e Dense layer with softmax activation for classification
    out = Dense(6, activation="softmax")(cls_token)

    # 117.f Keras model definition
    model = tf.keras.Model(
        inputs=[input_word_ids, attention_mask], outputs=out
    )

    # 117.g Compile the model
    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss="categorical_crossentropy",
        metrics=["accuracy", Precision(), Recall(), AUC()],
    )

    return model

In [ ]:
# 119. Load the Pre-Trained Model
transformer_layer = TFDistilBertModel.from_pretrained(
    "distilbert-base-multilingual-cased"
)

In [ ]:
# 120. Create the Model with the Pre-Trained Transformer Layers and Custom Layers for Fine-Tuning
model_v3 = create_model(transformer_layer, max_len=max_length)

In [ ]:
# 121. Display Model Summary
model_v3.summary()

# 122. Set the First Three Layers of the Model as Non-Trainable
model_v3.layers[0].trainable = False
model_v3.layers[1].trainable = False
model_v3.layers[2].trainable = False

In [ ]:
# 123. Display Updated Model Summary
model_v3.summary()

In [ ]:
# 124. Define Hyperparameters
n_steps = X_train_final.shape[0] // BATCH_SIZE  # Number of steps per epoch
num_epochs = 3  # Number of epochs

# 125. Train the Model
%%time
history = model_v3.fit(
    train_dataset,  # Training dataset
    steps_per_epoch=n_steps,  # Number of steps per epoch
    validation_data=valid_dataset,  # Validation dataset
    epochs=num_epochs  # Number of epochs
)

#### Model Implementation: Custom Tokenization Function

In [ ]:
# 126. Plot Learning Curves
loss, val_loss = history.history['loss'], history.history['val_loss']  # Extract training and validation loss

plt.plot(loss, label='Training Error')  # Plot training error
plt.plot(val_loss, label='Validation Error')  # Plot validation error

plt.legend()  # Add a legend
plt.show()  # Display the plot

In [ ]:
# 129. Extract Predicted Labels
predicted_labels = predictions.argmax(axis=1)

In [ ]:
# 130. Print Classification Report
print(classification_report(y_test_le, predicted_labels))

In [ ]:
# 131. Print Confusion Matrix
print(confusion_matrix(y_test_le, predicted_labels))

In [ ]:
# 132. Print Accuracy Score
print(accuracy_score(y_test_le, predicted_labels))

In [ ]:
# 133. Save the Model in TensorFlow Format
model_v3.save("model_v3.keras")

In [ ]:
# 134. Load the Model

# 134.a Imports
from transformers import TFDistilBertModel
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope

# 134.b Load the model with the custom layer registered
reloaded_model = load_model(
    "model_v3.keras",
    custom_objects={"TransformerLayer": TransformerLayer}
)

In [ ]:
# 135. New Sentence (Sentiment = Fear)
sentence = "i even feel a little shaky"

In [ ]:
# 136. Create a DataFrame with the Sentence
df_new = pd.DataFrame({'Sentence': [sentence]})

In [ ]:
# 137. Apply the Preprocessing Function
df_new['Processed_Sentence'] = df_new['Sentence'].apply(preprocess_text)

In [ ]:
# 138. Encode the New Data
new_data = encode_texts(
df_new['Processed_Sentence'], fast_tokenizer, maxlen=max_length)

In [ ]:
# 139. Make Predictions with the Loaded Model
predictions = model_v3.predict(new_data)

In [ ]:
# 140. Select the Class with the Highest Probability
highest_prob_class = np.argmax(predictions, axis=1)

# 141. Display the Class with the Highest Probability
highest_prob_class

In [ ]:
# 142. Get the Class Name
class_name = label_encoder_v3.inverse_transform(highest_prob_class)

In [ ]:
# 143. Predicted Class
class_name

In [ ]:
# 144. Disable Parallelism in the Transformers Package
%env TOKENIZERS_PARALLELISM=false